## CAPSTONE PROJECT:
## FIND THE KEY AREAS TO DO BUSINESS IN BANGALORE, INDIA

Downlaod all libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

Get data and load it onto a dataframe

In [2]:
df_bang = pd.read_csv("https://raw.githubusercontent.com/suvajit/opendata/master/BBMP/data/CSV/BBMPwards.csv")
df_bang.head()

,OBJECTID,ASS_CONST_,ASS_CONST1,WARD_NO,WARD_NAME,TOT_HH,POP_TOTAL,POP_M,POP_F,POP_SC,POP_ST,POP_LIT,POP_WORK,AREA_SQ_KM,LAT,LON,RESERVATIO
0,186,150,Yelahanka,1,Kempegowda Ward,8647,34783,18197,16586,2816,1097,27748,14794,10.47,13.116188,77.599714,Backward Category - B
1,1,150,Yelahanka,2,Chowdeswari Ward,9506,36602,19060,17542,3941,810,27160,16865,7.06,13.121709,77.580422,General
2,2,150,Yelahanka,3,Atturu,14605,58129,30799,27330,6480,1859,46738,23818,10.15,13.102805,77.560038,General (Women)
3,3,150,Yelahanka,4,Yelahanka Satellite Town,10583,41986,21799,20187,6319,1065,33599,17722,4.90,13.090987,77.583925,Backward Category - A
4,13,152,Byatarayanapura,5,Jakkuru,12387,52025,27269,24756,6423,973,37879,20445,23.96,13.096250,77.623314,General (Women)


In [3]:
list = ['OBJECTID','ASS_CONST_','ASS_CONST1','POP_M','POP_F','POP_SC','POP_ST','POP_LIT','RESERVATIO','AREA_SQ_KM', 'POP_WORK', 'POP_TOTAL']
df_bang = df_bang.drop(columns=list)
df_bang.head()

,WARD_NO,WARD_NAME,TOT_HH,LAT,LON
0,1,Kempegowda Ward,8647,13.116188,77.599714
1,2,Chowdeswari Ward,9506,13.121709,77.580422
2,3,Atturu,14605,13.102805,77.560038
3,4,Yelahanka Satellite Town,10583,13.090987,77.583925
4,5,Jakkuru,12387,13.096250,77.623314


Normalizing the TOT_HH column

In [78]:
df_bang["TOT_HH"] = df_bang["TOT_HH"] / df_bang["TOT_HH"].max()
df_bang.head()

,WARD_NO,Neighborhood,TOT_HH,LAT,LON
0,1,Kempegowda Ward,0.360307,13.116188,77.599714
1,2,Chowdeswari Ward,0.396100,13.121709,77.580422
2,3,Atturu,0.608567,13.102805,77.560038
3,4,Yelahanka Satellite Town,0.440977,13.090987,77.583925
4,5,Jakkuru,0.516147,13.096250,77.623314


In [5]:
df_bang = df_bang.rename(columns={'WARD_NAME': 'Neighborhood'})
df_bang.head()

,WARD_NO,Neighborhood,TOT_HH,LAT,LON
0,1,Kempegowda Ward,0.360307,13.116188,77.599714
1,2,Chowdeswari Ward,0.396100,13.121709,77.580422
2,3,Atturu,0.608567,13.102805,77.560038
3,4,Yelahanka Satellite Town,0.440977,13.090987,77.583925
4,5,Jakkuru,0.516147,13.096250,77.623314


In [92]:

!wget --quiet https://raw.githubusercontent.com/openbangalore/bangalore/master/bangalore/GIS/bangalore_pincode.json -o Bangalore_geojson
bang_geo = r'https://raw.githubusercontent.com/openbangalore/bangalore/master/bangalore/GIS/bangalore_pincode.json' # geojson file
latitude = 12.9716
longitude = 77.5946

# let Folium determine the scale.
bang_map = folium.Map(location=[latitude, longitude], zoom_start=11)
bang_map.choropleth(
    geo_data=bang_geo,
    data=df_bang,
    columns=['Neighborhood', 'TOT_HH'],
    key_on='feature.properties.name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Number of houses in Bangalore',
    reset=True
)
bang_map

From this we sort of get an idea on where the "BEST AREAS" are. But let's go further.

Define FOURSQUARE CREDENTIALS to get all the venue details from Foursquare API.

In [7]:
CLIENT_ID = 'TJVZLX2QAMYLD3PZJF22NBRD2QTGRKBKZSGVPMDWGQMLSBSN' # your Foursquare ID
CLIENT_SECRET = 'BGNZTAXHW0AM2DNPUCD1LEVOTPGN0P1AO5A2BY14JVYVXDY2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TJVZLX2QAMYLD3PZJF22NBRD2QTGRKBKZSGVPMDWGQMLSBSN
CLIENT_SECRET:BGNZTAXHW0AM2DNPUCD1LEVOTPGN0P1AO5A2BY14JVYVXDY2


In [9]:
CLIENT_ID = 'RXXZOONKT0JPQHHCSVDQBORKRX3OW1U2UTROFGGOXJZLHAX1' # your Foursquare ID
CLIENT_SECRET = 'HIKA4H5R3L3MPFCYUTOZINKQO0M4NHJ3HO2KS3WEELJXOEEM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RXXZOONKT0JPQHHCSVDQBORKRX3OW1U2UTROFGGOXJZLHAX1
CLIENT_SECRET:HIKA4H5R3L3MPFCYUTOZINKQO0M4NHJ3HO2KS3WEELJXOEEM


Create url

In [10]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=RXXZOONKT0JPQHHCSVDQBORKRX3OW1U2UTROFGGOXJZLHAX1&client_secret=HIKA4H5R3L3MPFCYUTOZINKQO0M4NHJ3HO2KS3WEELJXOEEM&v=20180605&ll=12.9716,77.5946&radius=500&limit=100'

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c358021db04f5568e964498'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bangalore',
  'headerFullLocation': 'Bangalore',
  'headerLocationGranularity': 'city',
  'totalResults': 51,
  'suggestedBounds': {'ne': {'lat': 12.976100004500005,
    'lng': 77.59920922318105},
   'sw': {'lat': 12.967099995499996, 'lng': 77.58999077681895}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bcd805cfb84c9b61512223e',
       'name': 'UB City',
       'location': {'address': 'at Vittal Mallya Rd',
        'lat': 12.971814708809388,
        'lng': 77.5958776473999,
        'labeledLatLngs': [{'label': 'display',
          'lat': 12.971814708809388,
       

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json file and put it into a *pandas* dataframe.

In [13]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,UB City,Shopping Mall,12.971815,77.595878
1,Toscano,Italian Restaurant,12.971980,77.596066
2,Skyye,Lounge,12.971646,77.596242
3,J W Kitchen,Asian Restaurant,12.972410,77.594592
4,Café Noir,French Restaurant,12.971995,77.596001


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

51 venues were returned by Foursquare.


Creating a function to repeat the same process to all the neighboring areas in Bangalore

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get all neighboring areas

In [18]:
bangalore_venues = getNearbyVenues(names=df_bang['Neighborhood'],
                                   latitudes=df_bang['LAT'],
                                   longitudes=df_bang['LON']
                                  )

Kempegowda Ward
Chowdeswari Ward
Atturu
Yelahanka Satellite Town
Jakkuru
Thanisandra
Byatarayanapura
Kodigehalli
Vidyaranyapura
Dodda Bommasandra
Kuvempu Nagar
Shettihalli
Mallasandra
Bagalakunte
T Dasarahalli
Jalahalli
J P Park
Radhakrishna Temple Ward
Sanjaya Nagar
Ganga Nagar
Hebbala
Vishwanath Nagenahalli
Nagavara
HBR Layout
Horamavu
Ramamurthy Nagar
Banasavadi
Kammanahalli
Kacharkanahalli
Kadugondanahalli
Kushal Nagar
Kaval Bairasandra
Manorayanapalya
Gangenahalli
Aramane Nagara
Mattikere
Yeshwanthpura
HMT Ward
Chokkasandra
Dodda Bidarakallu
Peenya Industrial Area
Lakshmi Devi Nagar
Nandini Layout
Marappana Palya
Malleswaram
Jayachamarajendra Nagar
Devara Jeevanahalli
Muneshwara Nagar
Lingarajapura
Benniganahalli
Vijnanapura
K R Puram
Basavanapura
Hudi
Devasandra
A Narayanapura
C V Raman Nagar
New Tippasandara
Maruthi Seva Nagar
Sagayarapuram
S K Garden
Ramaswamy Palya
Jayamahal
Rajamahal Guttahalli
Kadu Malleshwar Ward
Subramanya Nagar
Nagapura
Mahalakshimpuram
Laggere
Rajagopal 

In [19]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
A Narayanapura,5,5,5,5,5,5
Adugodi,2,2,2,2,2,2
Agrahara Dasarahalli,3,3,3,3,3,3
Anjanapura,1,1,1,1,1,1
Arakere,3,3,3,3,3,3
Aramane Nagara,4,4,4,4,4,4
Attiguppe,16,16,16,16,16,16
Atturu,2,2,2,2,2,2
Azad Nagar,4,4,4,4,4,4


In [20]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 180 uniques categories.


Now we start narrowing down on the "BEST AREAS".

Analyze each Neighborhood

In [24]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
cols = [bangalore_onehot.columns[-1]] + [col for col in bangalore_onehot if col != bangalore_onehot.columns[-1]]
bangalore_onehot = bangalore_onehot[cols]

bangalore_onehot.head()

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Nightclub,North Indian Restaurant,Other Nightlife,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Print Shop,Pub,Punjabi Restaurant,Radio Station,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Atturu,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Atturu,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Yelahanka Satellite Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Yelahanka Satellite Town,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [25]:
bangalore_onehot.shape

(1236, 181)

Grouping by neighborhood and taking the mean of the frequency of occurrence of each category

In [27]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped.head()

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Campground,Candy Store,Capitol Building,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cricket Ground,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Historic Site,Hookah Bar,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Multiplex,Museum,Music Venue,Nightclub,North Indian Restaurant,Other Nightlife,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Print Shop,Pub,Punjabi Restaurant,Radio Station,Rajasthani Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,A Narayanapura,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adugodi,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [28]:
bangalore_grouped.shape

(167, 181)

Printing each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in bangalore_grouped['Neighborhood']:
    temp = bangalore_grouped[bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
temp.shape

(180, 2)

Creating a function to sort the venues in descending order.

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a new dataframe to display the top 5 venues for each neighborhood.

In [109]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,A Narayanapura,Fish Market,Fast Food Restaurant,Movie Theater,Bus Station,Bus Stop
1,Adugodi,Café,Indian Restaurant,Zoo,Flower Shop,Flea Market
2,Agrahara Dasarahalli,Bagel Shop,Indian Restaurant,Café,Zoo,Flower Shop
3,Anjanapura,Campground,Zoo,Electronics Store,Flower Shop,Flea Market
4,Arakere,ATM,Badminton Court,Department Store,Food,Flower Shop
5,Aramane Nagara,Liquor Store,Playground,Pub,Tea Room,Zoo
6,Attiguppe,Seafood Restaurant,Breakfast Spot,Dessert Shop,Karnataka Restaurant,Juice Bar
7,Atturu,ATM,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market
8,Azad Nagar,Café,Multiplex,Fast Food Restaurant,Shopping Mall,Flea Market
9,BTM Layout,Indian Restaurant,Snack Place,Ice Cream Shop,Bakery,Café


Adding the total houses present per neighbrhood found earlier to this dataframe

In [33]:
neighborhoods_venues_sorted['TOT_HH'] = df_bang['TOT_HH']
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,TOT_HH
0,A Narayanapura,Fish Market,Fast Food Restaurant,Movie Theater,Bus Station,Bus Stop,0.360307
1,Adugodi,Café,Indian Restaurant,Zoo,Flower Shop,Flea Market,0.396100
2,Agrahara Dasarahalli,Bagel Shop,Indian Restaurant,Café,Zoo,Flower Shop,0.608567
3,Anjanapura,Campground,Zoo,Electronics Store,Flower Shop,Flea Market,0.440977
4,Arakere,ATM,Badminton Court,Department Store,Food,Flower Shop,0.516147


##  Clustering using K -means

In [99]:
# set number of clusters
kclusters = 4

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_#[0:10] 

array([1, 0, 0, 1, 1, 1, 1, 3, 0, 1, 1, 1, 0, 2, 2, 1, 1, 0, 2, 2, 3, 2,
       2, 1, 2, 1, 1, 2, 0, 1, 1, 1, 1, 1, 1, 2, 0, 3, 1, 2, 2, 1, 0, 2,
       1, 2, 2, 1, 2, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 0,
       2, 2, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 3, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 2, 2, 1, 0, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1,
       2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 3, 0, 1, 1, 1, 2, 1, 0, 1, 0, 1, 1,
       1, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 0, 1, 1, 1, 2, 1, 2, 1, 1, 3, 3,
       1, 1, 2, 1, 1, 2, 3, 1, 2, 1, 2, 0, 2], dtype=int32)

Creating a new Dataframe that contains the cluster labels and as well as the 

In [100]:
bangalore_merged = df_bang.merge(neighborhoods_venues_sorted, on='Neighborhood', how='left')
bangalore_merged = bangalore_merged.dropna(how='any').reset_index()
bangalore_merged['Neighborhood'] = bangalore_merged['Neighborhood'].sort_values(ascending=False)
bangalore_merged = bangalore_merged.drop('index', 1)
bangalore_merged['Cluster Labels'] = kmeans.labels_
bangalore_merged.head()
#bangalore_merged.head() # check the last columns!

,WARD_NO,Neighborhood,TOT_HH_x,LAT,LON,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,TOT_HH_y,Cluster Labels
0,3,Atturu,0.608567,13.102805,77.560038,ATM,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market,0.501521,1
1,4,Yelahanka Satellite Town,0.440977,13.090987,77.583925,Pizza Place,Outlet Store,Dessert Shop,Café,Eastern European Restaurant,0.270511,0
2,6,Thanisandra,0.686279,13.056525,77.622411,Sports Club,Zoo,Electronics Store,Flower Shop,Flea Market,0.455561,0
3,7,Byatarayanapura,0.778824,13.061071,77.597371,Department Store,Indian Restaurant,Print Shop,Bar,Zoo,0.512313,1
4,11,Kuvempu Nagar,0.354973,13.072080,77.541869,Fast Food Restaurant,Outdoors & Recreation,Zoo,Eastern European Restaurant,Flea Market,0.274970,1


In [101]:
import folium
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['LAT'], bangalore_merged['LON'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Cluster 1

In [102]:
cluster1 = bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0]
cluster1.shape

(25, 12)

Cluster 2

In [103]:
cluster2 = bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1]
cluster2.shape

(84, 12)

Cluster 3

In [104]:
cluster3 = bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2]
cluster3.shape

(50, 12)

Cluster 4

In [105]:
cluster4 = bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3]
cluster4.shape

(8, 12)

Cluster 5

As you can see Cluster 2 has the most number of Neigborhoods. From this we can conclude that the most visited places Hence, we pick the top 5 of this cluster.

In [106]:
Final_df = cluster2
Final_df = Final_df.iloc[0:5,:]
Final_df

,WARD_NO,Neighborhood,TOT_HH_x,LAT,LON,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,TOT_HH_y,Cluster Labels
0,3,Atturu,0.608567,13.102805,77.560038,ATM,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market,0.501521,1
3,7,Byatarayanapura,0.778824,13.061071,77.597371,Department Store,Indian Restaurant,Print Shop,Bar,Zoo,0.512313,1
4,11,Kuvempu Nagar,0.354973,13.072080,77.541869,Fast Food Restaurant,Outdoors & Recreation,Zoo,Eastern European Restaurant,Flea Market,0.274970,1
5,14,Bagalakunte,0.713238,13.058893,77.500280,Snack Place,Bakery,Zoo,Electronics Store,Flea Market,0.354973,1
6,15,T Dasarahalli,0.368724,13.045358,77.514736,Metro Station,Indian Sweet Shop,Resort,Karnataka Restaurant,Zoo,0.404975,1


Create a map and add the neighborhoods as markers on the map.

In [107]:
# create map and display it
bangalore_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# instantiate a feature group for the incidents in the dataframe
best_areas = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(Final_df.LAT, Final_df.LON):
    best_areas.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
bangalore_map.add_child(best_areas)